# MX Account LTV Analysis

Compute the 18mo and 50mo LTVs vs default rate reduction (relative %).

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
module_path

'/Users/kenny.liao/projects/pLTV'

In [4]:
import pandas as pd
import numpy as np
from pLTV.lib.pyltv import Model
from dbm import DBM
import plotly
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit

# save local javascript for render plotly visualizations
from IPython.display import Javascript
Javascript(
"""require.config({
 paths: { 
     plotly: 'https://cdn.plot.ly/plotly-latest.min'
 }
});"""
)

from IPython.display import Javascript
from plotly.offline import get_plotlyjs
Javascript(get_plotlyjs())

ModuleNotFoundError: No module named 'pLTV'

In [ ]:
mx_data = pd.read_csv('data/ltv_report/mx_5-12.csv')

## LTV vs Default

In [ ]:
market='mx'
forecast_method='powerslope'
min_months=5

times=[18, 50]

results = {}
for x in 1 - np.linspace(0, 0.2, 5):
    m = Model(mx_data, market=market, fcast_method=forecast_method, default_stress=x)
    
    ltvs = {}
    for t in times:
        # generate the forecast
        m.forecast = m.forecast_data(m.data, min_months=min_months, n_months=t)
        
        # get data from month t
        data_t = m.forecast.loc[t,:]
        
        # computed the weighted average default from t=0 to t=t
        m.forecast['defaulted_origination'] = m.forecast['default_rate_365dpd']*m.forecast.origination_per_original
        default = m.forecast.defaulted_origination.sum()/m.forecast.origination_per_original.sum()
        
        dcf_ltv = data_t['cumulative_dcf_ltv_per_original'].mean()
        ltv = data_t['cumulative_ltv_per_original'].mean()
        
        ltvs[t] = (default, ltv, dcf_ltv)
        
    results[round(1-x, 2)] = ltvs
        

## 18 mo LTV

In [ ]:
default=[]
dcf_ltvs_18 = []
ltvs_18 = []
for r in results:
    default.append(results[r][18][0])
    dcf_ltvs_18.append(results[r][18][2])
    ltvs_18.append(results[r][18][1])

In [ ]:
traces = [
    go.Scatter(name='18 mo LTV', x=list(results.keys()), y=ltvs_18),
    go.Scatter(name='18 mo DC LTV', x=list(results.keys()), y=dcf_ltvs_18)
]

fig = go.Figure(traces)
fig.update_layout(title='LTV vs Default Rate Reduction',
                 xaxis=dict(title='Default Rate Reduction (relative %)', tickformat='0.2%'),
                 yaxis=dict(title='LTV ($)'))
fig.show()

In [ ]:
# ltv = intercept + slope * dr

def ltv_to_dr18(ltv):
    slope = (ltvs_18[1] - ltvs_18[0])/5
    intercept = ltvs_18[0]
    return (ltv - intercept)/slope

def dc_ltv_to_dr18(ltv):
    slope = (dcf_ltvs_18[1] - dcf_ltvs_18[0])/5
    intercept = dcf_ltvs_18[0]
    return (ltv - intercept)/slope

## 50 mo LTV

In [ ]:
ltv_to_dr(46)

In [ ]:
default50=[]
dcf_ltvs_50 = []
ltvs_50 = []
for r in results:
    default50.append(results[r][50][0])
    dcf_ltvs_50.append(results[r][50][2])
    ltvs_50.append(results[r][50][1])

In [ ]:
traces2 = [
    go.Scatter(name='50 mo LTV', x=list(results.keys()), y=ltvs_50),
    go.Scatter(name='50 mo DC LTV', x=list(results.keys()), y=dcf_ltvs_50)
]

fig = go.Figure(traces2)
fig.update_layout(title='LTV vs Default Rate Reduction',
                 xaxis=dict(title='Default Rate Reduction (relative %)', tickformat='0.2%'),
                 yaxis=dict(title='LTV ($)'))
fig.show()

In [ ]:
# ltv = intercept + slope * dr

def ltv_to_dr50(ltv):
    slope = (ltvs_50[1] - ltvs_50[0])/5
    intercept = ltvs_50[0]
    return (ltv - intercept)/slope

def dc_ltv_to_dr50(ltv):
    slope = (dcf_ltvs_50[1] - dcf_ltvs_50[0])/5
    intercept = dcf_ltvs_50[0]
    return (ltv - intercept)/slope

---